# Pre-Processing: Housing Price Predicition

In the preprocessing phase we are going to accomplish two things. We are going to seperate the discrete and continuous features from one another, and scale the continuous features using StandardScaler() and one-hot encode the discrete features. We will then concatenate them back together into one dataset and save the preprocessed dataset for modeling. We will do this three seperate times with different combinations of features that we think may have an effect on the feature importance potion prior to modeling.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from library.sb_utils import save_file

In [69]:
pd.set_option('display.max_rows', 500)

In [70]:
data = pd.read_csv("Data Files/train_data_engineered.csv")

In [71]:
data.shape

(1137, 60)

In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137 entries, 0 to 1136
Data columns (total 60 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SalePrice      1137 non-null   int64  
 1   LotFrontage    1137 non-null   float64
 2   LotArea        1137 non-null   int64  
 3   MasVnrArea     1137 non-null   float64
 4   BsmtFinSF1     1137 non-null   int64  
 5   TotalBsmtSF    1137 non-null   int64  
 6   1stFlrSF       1137 non-null   int64  
 7   2ndFlrSF       1137 non-null   int64  
 8   GrLivArea      1137 non-null   int64  
 9   BsmtFullBath   1137 non-null   int64  
 10  FullBath       1137 non-null   int64  
 11  HalfBath       1137 non-null   int64  
 12  TotRmsAbvGrd   1137 non-null   int64  
 13  Fireplaces     1137 non-null   int64  
 14  GarageArea     1137 non-null   int64  
 15  WoodDeckSF     1137 non-null   int64  
 16  OpenPorchSF    1137 non-null   int64  
 17  MSZoning       1137 non-null   object 
 18  Alley   

Let's go ahead and drop some categorical variables based on common sense, such as Sale Condition not being a good predictor for home value before it is on the market.

In [73]:
data1 = data.drop(['SaleCondition','SaleType','Alley','Condition1','BldgType','Exterior1st','Exterior2nd','Foundation','Electrical','MSSubClass','MSZoning','BsmtFinType1','BsmtFinType2','Neighborhood'],axis=1)

In [74]:
data1.columns

Index(['SalePrice', 'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath',
       'FullBath', 'HalfBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'LotShape', 'LandContour', 'LotConfig',
       'LandSlope', 'HouseStyle', 'RoofStyle', 'MasVnrType', 'ExterQual',
       'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'Heating',
       'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive',
       'Fence', 'MiscFeature', 'OverallQual', 'OverallCond', 'House_Age',
       'Garage_Age'],
      dtype='object')

In [81]:
catfeatures = ['LotShape', 'LandContour', 'LotConfig',
       'LandSlope', 'HouseStyle', 'RoofStyle', 'MasVnrType',
       'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'Heating', 'HeatingQC', 'CentralAir',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'Fence',
       'MiscFeature']
numfeatures = data1.drop(columns=['SalePrice','LotShape', 'LandContour', 'LotConfig',
       'LandSlope', 'HouseStyle', 'RoofStyle', 'MasVnrType',
       'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'Heating', 'HeatingQC', 'CentralAir',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'Fence',
       'MiscFeature'])

In [82]:
dummies1 = pd.get_dummies(data1[catfeatures])
dummies1.head().T

,0,1,2,3,4
LotShape_IR1,0,0,1,1,0
LotShape_IR2,0,0,0,0,0
LotShape_IR3,0,0,0,0,0
LotShape_Reg,1,1,0,0,1
LandContour_Bnk,0,0,0,0,0
LandContour_HLS,0,0,0,0,0
LandContour_Low,0,0,0,0,0
LandContour_Lvl,1,1,1,1,1
LotConfig_Corner,0,0,0,0,0
LotConfig_CulDSac,0,0,0,0,0


In [83]:
scaler = StandardScaler()
scaler.fit(numfeatures)
scaled1 = scaler.transform(numfeatures)
scaled1 = pd.DataFrame(scaled1, index=numfeatures.index, columns=numfeatures.columns)

In [84]:
scaled1.head()

,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,TotRmsAbvGrd,Fireplaces,GarageArea,WoodDeckSF,OpenPorchSF,OverallQual,OverallCond,House_Age,Garage_Age
0,0.279857,-0.217666,0.888197,0.682376,-0.393202,-0.754347,1.344658,0.692021,1.167737,0.875418,1.27376,1.208270,-0.875318,0.460518,-0.820323,0.436367,0.762179,-0.488014,-1.024963,-0.821371
1,0.760132,0.078136,-0.619443,1.339277,0.679639,0.478393,-0.786663,-0.376695,-0.781921,0.875418,-0.73493,-0.203655,0.764281,0.024408,1.985010,-0.768407,0.011878,2.236397,-0.106294,0.219722
2,0.375912,0.502548,0.626668,0.151058,-0.224084,-0.560023,1.374606,0.873321,1.167737,0.875418,1.27376,-0.203655,0.764281,0.757865,-0.820323,0.061110,0.762179,-0.488014,-0.956913,-0.744253
3,0.888205,1.276779,2.072771,0.559207,0.370470,0.123145,1.841300,1.856159,1.167737,0.875418,1.27376,1.914232,0.764281,1.887784,0.987140,0.890627,1.512479,-0.488014,-0.922889,-0.705694
4,0.600040,0.202631,0.811277,2.283573,1.800044,1.790077,-0.786663,0.653853,1.167737,0.875418,-0.73493,0.502307,0.764281,0.896627,1.580214,0.357366,1.512479,-0.488014,-1.058988,-0.859930


In [91]:
data_preprocessed1 = pd.concat([scaled1,dummies1],axis=1)

In [92]:
data_preprocessed1 = pd.concat([data['SalePrice'],data_preprocessed1],axis=1)

In [93]:
data_preprocessed1.head()

,SalePrice,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,...,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,MiscFeature_None,MiscFeature_Shed
0,208500,0.279857,-0.217666,0.888197,0.682376,-0.393202,-0.754347,1.344658,0.692021,1.167737,...,0,0,1,0,0,0,0,1,1,0
1,181500,0.760132,0.078136,-0.619443,1.339277,0.679639,0.478393,-0.786663,-0.376695,-0.781921,...,0,0,1,0,0,0,0,1,1,0
2,223500,0.375912,0.502548,0.626668,0.151058,-0.224084,-0.560023,1.374606,0.873321,1.167737,...,0,0,1,0,0,0,0,1,1,0
3,250000,0.888205,1.276779,2.072771,0.559207,0.370470,0.123145,1.841300,1.856159,1.167737,...,0,0,1,0,0,0,0,1,1,0
4,307000,0.600040,0.202631,0.811277,2.283573,1.800044,1.790077,-0.786663,0.653853,1.167737,...,0,0,1,0,0,0,0,1,1,0


In [94]:
data_preprocessed1.shape

(1137, 138)

In [96]:
data_preprocessed1.tail(20)

,SalePrice,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,...,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,MiscFeature_None,MiscFeature_Shed
1117,186500,1.176370,0.248158,1.826626,-1.022670,-0.181805,-0.432499,1.284761,0.887635,1.167737,...,0,0,1,0,0,0,0,1,1,0
1118,160000,0.760132,2.084448,-0.619443,1.237844,0.320264,0.065455,-0.786663,-0.701127,1.167737,...,0,1,0,0,0,0,0,1,1,0
1119,174000,0.760132,-0.230527,1.203571,-1.022670,0.830259,1.313377,-0.786663,0.279325,1.167737,...,0,0,1,1,0,0,0,0,1,0
1120,120500,0.119766,-0.076195,-0.619443,0.465018,-0.372062,-0.730056,-0.786663,-1.326135,-0.781921,...,0,0,1,0,1,0,0,0,1,0
1121,149700,1.080315,-0.485946,-0.619443,0.426377,-0.245224,0.399449,-0.786663,-0.438719,1.167737,...,0,0,1,0,0,1,0,0,1,0
1122,149300,-1.801332,-1.252717,0.511287,0.660640,-0.414342,-0.778637,-0.786663,-1.364304,1.167737,...,0,0,1,0,0,0,0,1,1,0
1123,310000,0.920223,0.439015,0.611284,0.824865,0.032235,-0.238175,1.661611,1.400523,1.167737,...,0,0,1,0,0,0,0,1,1,0
1124,121000,-1.801332,-0.113749,-0.619443,-1.022670,-0.139525,-0.462862,-0.786663,-1.116209,-0.781921,...,0,1,0,0,0,0,0,1,1,0
1125,179600,0.215821,-0.204805,0.195914,-1.022670,1.102433,0.964202,-0.786663,0.004989,-0.781921,...,0,0,1,0,0,0,0,1,1,0
1126,157900,-1.801332,4.333060,0.834353,0.409472,0.484096,0.253706,-0.786663,-0.553224,-0.781921,...,0,1,0,0,0,0,0,1,1,0


In [97]:
#Saving preprocessed data to new csv
datapath = 'C:\Springboard_\CapstoneTwo\Data Files'
save_file(data_preprocessed1, 'data_preprocessed.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)y
Writing file.  "C:\Springboard_\CapstoneTwo\Data Files\data_preprocessed.csv"


## Second Dataset with correlated independent variables removed

In [15]:
data2 = data.drop(['TotRmsAbvGrd','GrLivArea','LotFrontage'], axis=1)

In [16]:
data2 = data2.drop(['SaleCondition','SaleType','Alley','Condition1','BldgType','Exterior1st','Exterior2nd','Foundation','Electrical','MSSubClass','MSZoning','BsmtFinType1','BsmtFinType2','Neighborhood'],axis=1)

In [17]:
data2.columns

Index(['SalePrice', 'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'BsmtFullBath', 'FullBath',
       'HalfBath', 'Fireplaces', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'HouseStyle',
       'RoofStyle', 'MasVnrType', 'ExterQual', 'ExterCond', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'Heating', 'HeatingQC', 'CentralAir',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'Fence',
       'MiscFeature', 'OverallQual', 'OverallCond', 'House_Age', 'Garage_Age'],
      dtype='object')

In [19]:
features = ['BsmtFullBath','FullBath', 'HalfBath', 'Fireplaces','LotShape', 'LandContour', 'LotConfig',
       'LandSlope', 'HouseStyle', 'RoofStyle', 'MasVnrType',
       'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'Heating', 'HeatingQC', 'CentralAir',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'Fence',
       'MiscFeature', 'OverallQual', 'OverallCond', 'House_Age',
       'Garage_Age']
dummies = pd.get_dummies(data2[features])
data_dummies = pd.concat([data2,dummies],axis=1)
data_encoded = data_dummies.drop(['BsmtFullBath','FullBath', 'HalfBath', 'Fireplaces','LotShape', 'LandContour', 'LotConfig',
       'LandSlope', 'HouseStyle', 'RoofStyle', 'MasVnrType',
       'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'Heating', 'HeatingQC', 'CentralAir',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'Fence',
       'MiscFeature', 'OverallQual', 'OverallCond', 'House_Age',
       'Garage_Age'], axis=1)

In [21]:
data_encoded.shape

(1137, 128)

In [22]:
#Saving preprocessed data to new csv
datapath = 'C:\Springboard_\CapstoneTwo\Data Files'
save_file(data_encoded, 'data_preprocessed2.csv', datapath)

Writing file.  "C:\Springboard_\CapstoneTwo\Data Files\data_preprocessed2.csv"


## Third Dataset removing the opposite side of the correlated variables

In [23]:
data3 = data.drop(['1stFlrSF','2ndFlrSF','LotFrontage'], axis=1)

In [24]:
data3 = data3.drop(['SaleCondition','SaleType','Alley','Condition1','BldgType','Exterior1st','Exterior2nd','Foundation','Electrical','MSSubClass','MSZoning','BsmtFinType1','BsmtFinType2','Neighborhood'],axis=1)

In [25]:
data3.columns

Index(['SalePrice', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'TotalBsmtSF',
       'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'LotShape',
       'LandContour', 'LotConfig', 'LandSlope', 'HouseStyle', 'RoofStyle',
       'MasVnrType', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'Fence', 'MiscFeature', 'OverallQual',
       'OverallCond', 'House_Age', 'Garage_Age'],
      dtype='object')

In [27]:
features = ['BsmtFullBath','FullBath', 'HalfBath', 'Fireplaces','LotShape', 'LandContour', 'LotConfig',
       'LandSlope', 'HouseStyle', 'RoofStyle', 'MasVnrType',
       'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'Heating', 'HeatingQC', 'CentralAir',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'Fence',
       'MiscFeature', 'OverallQual', 'OverallCond', 'House_Age',
       'Garage_Age']
dummies = pd.get_dummies(data3[features])
data_dummies = pd.concat([data3,dummies],axis=1)
data_encoded = data_dummies.drop(['BsmtFullBath','FullBath', 'HalfBath', 'Fireplaces','LotShape', 'LandContour', 'LotConfig',
       'LandSlope', 'HouseStyle', 'RoofStyle', 'MasVnrType',
       'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'Heating', 'HeatingQC', 'CentralAir',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'Fence',
       'MiscFeature', 'OverallQual', 'OverallCond', 'House_Age',
       'Garage_Age'], axis=1)

In [28]:
data_encoded.shape

(1137, 127)

In [29]:
#Saving preprocessed data to new csv
datapath = 'C:\Springboard_\CapstoneTwo\Data Files'
save_file(data_encoded, 'data_preprocessed3.csv', datapath)

Writing file.  "C:\Springboard_\CapstoneTwo\Data Files\data_preprocessed3.csv"
